In [4]:
!apt-get install -y mpich

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
mpich is already the newest version (4.0-3).
0 upgraded, 0 newly installed, 0 to remove and 34 not upgraded.


In [18]:
%%writefile hello_mpi.cpp
#include <mpi.h>
#include <iostream>
#include <vector>
#include <algorithm>
using namespace std;

void printArray(const int* arr, int size, const string& label) {
    cout << label << ": ";
    for (int i = 0; i < size; ++i)
        cout << arr[i] << " ";
    cout << endl;
}

int main(int argc, char** argv) {
    int rank, size;
    const int total_size = 8;
    int full_array[total_size] = { 23, 12, 4, 56, 9, 1, 44, 17 };
    vector<int> local_array;

    MPI_Init(&argc, &argv);
    MPI_Comm_rank(MPI_COMM_WORLD, &rank);
    MPI_Comm_size(MPI_COMM_WORLD, &size);

    int chunk_size = total_size / size;
    local_array.resize(chunk_size);

    // Print original array before scattering
    if (rank == 0) {
        printArray(full_array, total_size, "Unsorted array");
    }

    // Scatter the array to all processes
    MPI_Scatter(full_array, chunk_size, MPI_INT,
                local_array.data(), chunk_size, MPI_INT,
                0, MPI_COMM_WORLD);

    // Sort the local chunk
    sort(local_array.begin(), local_array.end());

    // Odd-even transposition sort
    for (int phase = 0; phase < size; ++phase) {
        if ((phase % 2 == 0 && rank % 2 == 0) || (phase % 2 == 1 && rank % 2 == 1)) {
            if (rank + 1 < size) {
                vector<int> neighbor_array(chunk_size);
                MPI_Sendrecv(local_array.data(), chunk_size, MPI_INT, rank + 1, 0,
                             neighbor_array.data(), chunk_size, MPI_INT, rank + 1, 0,
                             MPI_COMM_WORLD, MPI_STATUS_IGNORE);

                vector<int> merged(chunk_size * 2);
                merge(local_array.begin(), local_array.end(),
                      neighbor_array.begin(), neighbor_array.end(), merged.begin());

                copy(merged.begin(), merged.begin() + chunk_size, local_array.begin());
            }
        }
        else if (rank - 1 >= 0) {
            vector<int> neighbor_array(chunk_size);
            MPI_Sendrecv(local_array.data(), chunk_size, MPI_INT, rank - 1, 0,
                         neighbor_array.data(), chunk_size, MPI_INT, rank - 1, 0,
                         MPI_COMM_WORLD, MPI_STATUS_IGNORE);

            vector<int> merged(chunk_size * 2);
            merge(neighbor_array.begin(), neighbor_array.end(),
                  local_array.begin(), local_array.end(), merged.begin());

            copy(merged.begin() + chunk_size, merged.end(), local_array.begin());
        }
    }

    // Gather sorted chunks to full_array at rank 0
    MPI_Gather(local_array.data(), chunk_size, MPI_INT,
               full_array, chunk_size, MPI_INT,
               0, MPI_COMM_WORLD);

    if (rank == 0) {
        cout << "Final Sorted Array:\n";
        for (int i = 0; i < total_size; ++i)
            cout << full_array[i] << " ";
        cout << endl;
    }

    MPI_Finalize();
    return 0;
}


Overwriting hello_mpi.cpp


In [19]:
!mpic++ hello_mpi.cpp -o hello_mpi

In [21]:
!OMPI_ALLOW_RUN_AS_ROOT=1 OMPI_ALLOW_RUN_AS_ROOT_CONFIRM=1 mpirun --oversubscribe -np 4 ./hello_mpi

Unsorted array: 23 12 4 56 9 1 44 17 
Final Sorted Array:
1 4 9 12 17 23 44 56 
